In [31]:
import numpy as np
import pandas as pd
import statistics
import scipy.fftpack as fftpk
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
from scipy.io import wavfile
from scipy.fft import fft
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from google.colab import drive

In [32]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/Projeto/samples/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
def extract_signal_attributes(FFT_signal):
  min_signal = min(FFT_signal)
  max_signal = max(FFT_signal)
  mean_signal = FFT_signal.mean()
  power_signal = np.power(FFT_signal, 2)
  power_mean_signal = power_signal.mean()
  energy_signal = np.sum(np.abs(power_signal))
  variance_signal = statistics.variance(FFT_signal)
  rms_signal = np.sqrt(power_mean_signal)
  shape_factor_signal = rms_signal / mean_signal
  crest_factor_signal = max_signal / rms_signal

  return [min_signal, max_signal, mean_signal, power_mean_signal, energy_signal, variance_signal, rms_signal, shape_factor_signal, crest_factor_signal]

In [34]:
signals = pd.DataFrame(columns=['min_signal', 'max_signal', 'mean_signal', 'power_mean_signal', 'energy_signal', 'variance_signal', 'rms_signal', 'shape_factor_signal', 'crest_factor_signal'])

In [35]:
for x in range(1, 140):
  s_rate, signal_bad = wavfile.read(path + 'bad/' + str(x) + '.wav')
  FFT_signal_bad = abs(fft(signal_bad))
  signal_attributes = extract_signal_attributes(FFT_signal_bad)
  signals = signals.append(
      {
        'min_signal': signal_attributes[0], 
        'max_signal': signal_attributes[1], 
        'mean_signal': signal_attributes[2], 
        'power_mean_signal': signal_attributes[3], 
        'energy_signal': signal_attributes[4], 
        'variance_signal': signal_attributes[5], 
        'rms_signal': signal_attributes[6], 
        'shape_factor_signal': signal_attributes[7], 
        'crest_factor_signal': signal_attributes[8], 
        'label': 0
      },
      ignore_index=True
)

In [36]:
for x in range(1, 140):
  s_rate, signal_good = wavfile.read(path + 'good/' + str(x) + '.wav')
  FFT_signal_good = abs(fft(signal_good))
  signal_attributes = extract_signal_attributes(FFT_signal_good)
  signals = signals.append(
      {
        'min_signal': signal_attributes[0], 
        'max_signal': signal_attributes[1], 
        'mean_signal': signal_attributes[2], 
        'power_mean_signal': signal_attributes[3], 
        'energy_signal': signal_attributes[4], 
        'variance_signal': signal_attributes[5], 
        'rms_signal': signal_attributes[6], 
        'shape_factor_signal': signal_attributes[7], 
        'crest_factor_signal': signal_attributes[8], 
        'label': 1
      },
      ignore_index=True
)

In [37]:
signals.head()

,min_signal,max_signal,mean_signal,power_mean_signal,energy_signal,variance_signal,rms_signal,shape_factor_signal,crest_factor_signal,label
0,261.593540,6.614793e+06,253731.647656,2.779441e+11,4.891817e+15,2.135765e+11,527204.067003,2.077802,12.546931,0.0
1,383.810316,4.161230e+06,255112.639661,2.527505e+11,4.448408e+15,1.876787e+11,502742.940495,1.970670,8.277053,0.0
2,125.901187,5.135813e+06,256039.044107,2.572738e+11,4.528019e+15,1.917287e+11,507221.674468,1.981033,10.125381,0.0
3,122.435798,4.711181e+06,237846.896443,2.332358e+11,4.104950e+15,1.766747e+11,482944.925175,2.030487,9.755109,0.0
4,96.153185,4.725080e+06,232219.850190,2.183523e+11,3.843000e+15,1.644355e+11,467281.768345,2.012239,10.111843,0.0


In [38]:
signals.describe()

,min_signal,max_signal,mean_signal,power_mean_signal,energy_signal,variance_signal,rms_signal,shape_factor_signal,crest_factor_signal,label
count,278.000000,2.780000e+02,278.000000,2.780000e+02,2.780000e+02,2.780000e+02,278.000000,278.000000,278.000000,278.000000
mean,267.084355,1.321415e+07,308111.861187,5.871291e+11,1.033347e+16,4.907757e+11,754658.984120,2.438104,17.311829,0.500000
std,153.254893,4.065888e+06,38125.949630,1.980556e+11,3.485778e+15,1.770800e+11,132975.654517,0.214682,3.864150,0.500902
min,3.000000,4.161230e+06,227565.382855,2.183523e+11,3.843000e+15,1.644355e+11,467281.768345,1.905868,7.847177,0.000000
25%,149.443581,1.064347e+07,277866.706642,4.231101e+11,7.446738e+15,3.413169e+11,650469.057784,2.282473,14.475489,0.000000
50%,246.458668,1.277078e+07,309626.738713,5.896183e+11,1.037728e+16,4.891140e+11,767865.617770,2.466447,17.422157,0.500000
75%,361.810051,1.605640e+07,337392.813446,7.642043e+11,1.345000e+16,6.446800e+11,874187.067221,2.594408,20.355511,1.000000
max,764.367434,2.215960e+07,400943.564252,9.998711e+11,1.759773e+16,8.890124e+11,999935.560512,3.002535,30.263966,1.000000


In [39]:
signals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   min_signal           278 non-null    float64
 1   max_signal           278 non-null    float64
 2   mean_signal          278 non-null    float64
 3   power_mean_signal    278 non-null    float64
 4   energy_signal        278 non-null    float64
 5   variance_signal      278 non-null    float64
 6   rms_signal           278 non-null    float64
 7   shape_factor_signal  278 non-null    float64
 8   crest_factor_signal  278 non-null    float64
 9   label                278 non-null    float64
dtypes: float64(10)
memory usage: 21.8 KB


In [40]:
s_rate, signal_bad = wavfile.read(path + 'bad/1.wav')
s_rate, signal_good = wavfile.read(path + 'good/1.wav')

In [41]:
trace_bad = go.Scatter(
    x = np.arange(0, len(signal_bad), 1), 
    y = signal_bad,
    mode = 'lines',
    name = 'Sound Signal Bad'
)

trace_good = go.Scatter(
    x = np.arange(0, len(signal_good), 1), 
    y = signal_good,
    mode = 'lines',
    name = 'Sound Signal Good'
)

data = [trace_bad, trace_good]

layout = go.Layout(
    title = 'Sound Graphic',
    xaxis = {'title': 'Y'},
    yaxis = {'title': 'X'}
)

figure = go.Figure(data = data, layout = layout)

figure

In [42]:
FFT_signal_bad = abs(fft(signal_bad))
f_signal_bad = fftpk.fftfreq(len(FFT_signal_bad), (1 / s_rate))

FFT_signal_good = abs(fft(signal_good))
f_signal_good = fftpk.fftfreq(len(FFT_signal_good), (1 / s_rate))

trace_bad = go.Scatter(
    x = f_signal_bad[range(len(FFT_signal_bad)//2)], 
    y = FFT_signal_bad[range(len(FFT_signal_bad)//2)],
    mode = 'lines',
    name = 'Sound Signal Bad'
)

trace_good = go.Scatter(
    x = f_signal_good[range(len(FFT_signal_good)//2)], 
    y = FFT_signal_good[range(len(FFT_signal_good)//2)],
    mode = 'lines',
    name = 'Sound Signal Good'
)

data = [trace_bad, trace_good]

layout = go.Layout(
    title = 'Sound Graphic',
    xaxis = {'title': 'Frequency'},
    yaxis = {'title': 'Amplitude'}
)

figure = go.Figure(data = data, layout = layout)

figure

In [43]:
correlation = signals.corr(method='pearson')
correlation.style.background_gradient(cmap='coolwarm')

,min_signal,max_signal,mean_signal,power_mean_signal,energy_signal,variance_signal,rms_signal,shape_factor_signal,crest_factor_signal,label
min_signal,1.000000,0.156591,0.345723,0.270798,0.270798,0.256430,0.260590,0.034374,-0.006803,0.216497
max_signal,0.156591,1.000000,0.514646,0.730510,0.730510,0.749756,0.740403,0.800015,0.845582,0.462492
mean_signal,0.345723,0.514646,1.000000,0.904723,0.904723,0.879371,0.902857,0.423038,0.044244,0.795442
power_mean_signal,0.270798,0.730510,0.904723,1.000000,1.000000,0.998386,0.996685,0.752682,0.271148,0.863559
energy_signal,0.270798,0.730510,0.904723,1.000000,1.000000,0.998386,0.996685,0.752682,0.271148,0.863559
variance_signal,0.256430,0.749756,0.879371,0.998386,0.998386,1.000000,0.995413,0.786639,0.298599,0.860316
rms_signal,0.260590,0.740403,0.902857,0.996685,0.996685,0.995413,1.000000,0.769063,0.292237,0.857447
shape_factor_signal,0.034374,0.800015,0.423038,0.752682,0.752682,0.786639,0.769063,1.000000,0.566712,0.612310
crest_factor_signal,-0.006803,0.845582,0.044244,0.271148,0.271148,0.298599,0.292237,0.566712,1.000000,-0.011550
label,0.216497,0.462492,0.795442,0.863559,0.863559,0.860316,0.857447,0.612310,-0.011550,1.000000


In [44]:
cols = ['min_signal', 'max_signal', 'mean_signal','power_mean_signal', 'energy_signal', 'variance_signal', 'rms_signal', 'shape_factor_signal', 'crest_factor_signal']
signals[cols] = signals[cols].apply(minmax_scale)

In [45]:
x = signals[['mean_signal', 'power_mean_signal', 'energy_signal', 'variance_signal', 'rms_signal']]

In [46]:
y = signals['label']

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [48]:
pipeline = Pipeline([('classifier', SVC())])

In [60]:
search_space = [
  {'classifier': [KNeighborsClassifier()], 'classifier__n_neighbors': [1, 5, 10, 50]},
  {'classifier': [DecisionTreeClassifier(random_state = 1)], 'classifier__criterion': ['gini', 'entropy']},
  {'classifier': [SVC(random_state = 1)], 'classifier__C': [1, 5, 10, 50]}
]

In [61]:
base_search_CV = GridSearchCV(pipeline, search_space, cv=3)

In [62]:
base_search_CV_fit = base_search_CV.fit(x_train, y_train)

In [63]:
best_model = base_search_CV_fit.best_estimator_

In [64]:
best_model.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('classifier',
                 SVC(C=50, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [54]:
best_model_predict = best_model.predict(x_test)

In [55]:
accuracy_score(y_test, best_model_predict)

0.9761904761904762

In [56]:
recall_score(y_test, best_model_predict) 

0.9736842105263158

In [57]:
precision_score(y_test, best_model_predict)

0.9736842105263158

In [58]:
f1_score(y_test, best_model_predict)

0.9736842105263158

In [59]:
confusion_matrix(y_test, best_model_predict)

array([[45,  1],
       [ 1, 37]])